# Sample Notebook to demo GeoSpatial Reasoning with Functions.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
import os
import random
import uuid

import shapely
from faker import Faker
from gait import MAO, Agent, ObserverLoguru, Scratchpad
from rich.pretty import pprint
from shapely import distance
from shapely.geometry import Point

### Create a faker to fake cities, lat/lon values, etc...

In [ ]:
fake = Faker()

## Start defining the skills or functions that the agent will need.

In [ ]:
def geometry_intersect(
    geom1: str,
    geom2: str,
    scratchpad: Scratchpad,
) -> str:
    """Check if two geometries references by their UUIDs intersect.
    Return 'intersect' if they do, 'do not intersect' otherwise.

    :param geom1: The first geometry UUID.
    :param geom2: The second geometry UUID.
    :param scratchpad: Inject an instance of a Scratchpad at runtime.
    """
    geom_a = scratchpad[geom1]
    geom_b = scratchpad[geom2]
    return "intersect" if shapely.intersects(geom_a, geom_b) else "do not intersect"

In [ ]:
def geometry_intersection(
    geom1: str,
    geom2: str,
    scratchpad: Scratchpad,
) -> str:
    """Calculate the intersection of two geometries referenced by their UUIDs.
    Return the intersection geometry UUID.

    :param geom1: The first geometry UUID.
    :param geom2: The second geometry UUID.
    :param scratchpad: Inject an instance of a Scratchpad at runtime.
    """
    geom_a = scratchpad[geom1]
    geom_b = scratchpad[geom2]
    geom_i = shapely.intersection(geom_a, geom_b)

    geom_uuid = uuid.uuid1().hex
    scratchpad[geom_uuid] = geom_i
    return geom_uuid

In [ ]:
def get_current_temperature(
    location: str,
    unit: str,
) -> dict:
    """Get the current temperature at a location.

    :param location: The location. Can be a place, city, state, zipcode, country.
    :param unit: The temperature unit. Can be either 'Fahrenheit' or 'Celsius'. Default is 'Celsius'.
    """
    temp = random.uniform(-5.0, 40.0)
    unit = {"fahrenheit": "F", "celsius": "C"}.get(unit.lower(), "C")
    return {
        "location": location,
        "temperature": f"{temp:.1f}{chr(176)}{unit}",
    }

In [ ]:
def get_geometry_for_location(
    location: str,
    scratchpad: Scratchpad,
) -> str:
    """Get the geometry UUID of a location.

    :param location: The location. Can be a place, city, state, zipcode, country.
    :param scratchpad: Inject an instance of a Scratchpad at runtime.
    """
    if location not in scratchpad:
        # Call a TRUE geocoder here, but we are faking it for now.
        lon = float(fake.longitude())
        lat = float(fake.latitude())
        geom_uuid = uuid.uuid4().hex
        scratchpad[geom_uuid] = Point(lon, lat)
        scratchpad[location] = geom_uuid

    return scratchpad[location]

In [ ]:
def distance_in_meters(
    geom1: str,
    geom2: str,
    scratchpad: Scratchpad,
) -> dict:
    """Get the distance in meters between two geometries referenced by their UUID.

    :param geom1: The first geometry UUID.
    :param geom2: The second geometry UUID.
    :param scratchpad: Inject an instance of a Scratchpad at runtime.
    """
    point1 = scratchpad[geom1]
    point2 = scratchpad[geom2]
    meters = distance(point1, point2)
    return {"meters": meters}

## Define the system prompt.

In [ ]:
instructions = """You are an AI expert in geo-spatial data analysis with access to geo-spatial tools.
You run in a loop. At the end of the loop you output an answer.

Here are the rules you should always follow to solve your task:
- ALWAYS use the right arguments for the tools. Never use variable names, use the values instead.
- NEVER re-do an action call that you previously did with the exact same arguments.
- ALWAYS suffix the final answer with '<Answer/>'

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000."""

## Define the agent - Note here we can have OpenAI, Azure, Antropic, Bedrock, etc...

In [ ]:
model = "gpt-4.1"
azure_url = os.environ["AZURE_API_URL"]

In [ ]:
agent = Agent(
    model=f"azure/{model}",
    base_url=f"{azure_url}/{model}",
    instructions=instructions,
    functions=[
        geometry_intersect,
        geometry_intersection,
        get_current_temperature,
        get_geometry_for_location,
        distance_in_meters,
    ],
    temperature=0.0,
    stop=["<Answer/>"],
)

## Define the Multi Agent Orchestrator (MOA).

In [ ]:
mao = MAO(
    agent,
    observer=ObserverLoguru(),
)

## Let's start asking questions :-)

In [ ]:
city1 = fake.city()
city2 = fake.city()

for _ in mao(
    f"What is the temperature in {city1} and in {city2} and what is the distance between them?"
):
    if _.content:
        moa.terminate()

## Let's see the content of the "memory" of the agent.

In [ ]:
for k, v in mao.scratchpad:
    print(k, v)

## Let's see the "history" of the agent.

In [ ]:
for _ in mao.dialog:
    pprint(_, expand_all=True)